In [3]:
from delta import  # Delta Lake 라이브러리 임포트 *
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 *
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 *

In [4]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("data-skipping-delta-table")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [5]:
%load_ext sparksql_magic
%config SparkSql.limit=20

In [6]:
# 생성 some sample data frames
# A large data frame with 1 million rows
df = (spark.range(0, 1000000)
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"salary", 100*(rand() * 100).cast("int"))
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"gender", when((rand() * 2).cast("int") == 0, "M").otherwise("F"))
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"country_code", 
                        when(  # 조건문 시작(rand() * 4).cast("int") == 0, "US")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 1, "CN")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 2, "IN")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 3, "BR")
                        .otherwise(  # 기본값 설정'RU')))
df.show(5)

+---+------+------+------------+
| id|salary|gender|country_code|
+---+------+------+------------+
|  0|  4400|     M|          IN|
|  1|  3100|     M|          US|
|  2|  4700|     F|          US|
|  3|  2600|     M|          BR|
|  4|  8400|     F|          RU|
+---+------+------+------------+
only showing top 5 rows



In [7]:
(df.write
 .format("delta")  # Delta Lake 형식으로 저장
 .mode("overwrite")  # 기존 데이터 덮어쓰기
 .save("../data/tmp/employee_salary"))

In [8]:
df = (spark.range(0, 1000)
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"salary", 100*(rand() * 100).cast("int"))
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"gender", when((rand() * 2).cast("int") == 0, "M").otherwise("F"))
            .withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정"country_code", 
                        when(  # 조건문 시작(rand() * 4).cast("int") == 0, "US")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 1, "CN")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 2, "IN")
                        .when(  # 조건문 시작(rand() * 4).cast("int") == 3, "BR")
                        .otherwise(  # 기본값 설정'RU')))
(df.write
 .format("delta")  # Delta Lake 형식으로 저장
 .mode("append")  # 기존 데이터에 추가
 .save("../data/tmp/employee_salary"))

In [14]:
%%sparksql
DESCRIBE HISTORY delta.`/opt/workspace/data/tmp/employee_salary`

In [15]:
%%sparksql
OPTIMIZE delta.`/opt/workspace/data/tmp/employee_salary`

In [18]:
spark.conf.set(  # Spark 설정 변경"spark.databricks.delta.retentionDurationCheck.enabled", "false")

In [19]:
%%sparksql
VACUUM delta.`/opt/workspace/data/tmp/employee_salary` RETAIN 0 HOURS

Deleted 4 files and directories in a total of 1 directories.


In [22]:
spark.stop()  # Spark 세션 종료 - 리소스 정리